In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
links: pd.DataFrame = pd.read_csv('../DataSet/links.csv')
movies: pd.DataFrame = pd.read_csv('../DataSet/movies.csv')
ratings: pd.DataFrame = pd.read_csv('../DataSet/ratings.csv')
tags: pd.read_csv = pd.read_csv('../DataSet/tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


In [7]:
ratings.groupby('movieId')['rating'].mean()

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
            ...   
161944    5.000000
162376    4.500000
162542    5.000000
162672    3.000000
163949    5.000000
Name: rating, Length: 9066, dtype: float64

In [8]:
ratings[ratings['movieId']==1]

,userId,movieId,rating,timestamp
495,7,1,3.0,851866703
699,9,1,4.0,938629179
889,13,1,5.0,1331380058
962,15,1,2.0,997938310
3105,19,1,3.0,855190091
...,...,...,...,...
98531,660,1,2.5,1436680062
98714,663,1,4.0,1438397999
98740,664,1,3.5,1362421730
99858,670,1,4.0,938782344


In [9]:
# tags.groupby('movieId').reset_index().columns.values
print(type(tags.groupby('movieId').mean()))

<class 'pandas.core.frame.DataFrame'>


### 电影字典 & 写入基本信息

In [10]:
mv = {} # key:movieId,value:信息字典
for i in range(movies.shape[0]):
    index = movies.iloc[i,0]
    mv[index] = {'title':movies.iloc[i,1],'genres':movies.iloc[i,2]}
print(mv)

{1: {'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy'}, 2: {'title': 'Jumanji (1995)', 'genres': 'Adventure|Children|Fantasy'}, 3: {'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance'}, 4: {'title': 'Waiting to Exhale (1995)', 'genres': 'Comedy|Drama|Romance'}, 5: {'title': 'Father of the Bride Part II (1995)', 'genres': 'Comedy'}, 6: {'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller'}, 7: {'title': 'Sabrina (1995)', 'genres': 'Comedy|Romance'}, 8: {'title': 'Tom and Huck (1995)', 'genres': 'Adventure|Children'}, 9: {'title': 'Sudden Death (1995)', 'genres': 'Action'}, 10: {'title': 'GoldenEye (1995)', 'genres': 'Action|Adventure|Thriller'}, 11: {'title': 'American President, The (1995)', 'genres': 'Comedy|Drama|Romance'}, 12: {'title': 'Dracula: Dead and Loving It (1995)', 'genres': 'Comedy|Horror'}, 13: {'title': 'Balto (1995)', 'genres': 'Adventure|Animation|Children'}, 14: {'title': 'Nixon (1995)', 'genres': 'Drama'}, 15: {'tit

### 统计每个电影的tag

In [11]:
g = tags.groupby('movieId')
for k,v in g:
    # print(v)
    mvk = v.iloc[0,1]
    mvv = v.iloc[:,2].values
    # print(v.iloc[0,1],v.iloc[:,2].values)
    if mvk not in mv:
        mv[mvk]=dict()
    mv[mvk]['tag'] = list(mvv)
print(mv)

{1: {'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'tag': ['Pixar']}, 2: {'title': 'Jumanji (1995)', 'genres': 'Adventure|Children|Fantasy'}, 3: {'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance'}, 4: {'title': 'Waiting to Exhale (1995)', 'genres': 'Comedy|Drama|Romance'}, 5: {'title': 'Father of the Bride Part II (1995)', 'genres': 'Comedy', 'tag': ['steve martin']}, 6: {'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller'}, 7: {'title': 'Sabrina (1995)', 'genres': 'Comedy|Romance'}, 8: {'title': 'Tom and Huck (1995)', 'genres': 'Adventure|Children'}, 9: {'title': 'Sudden Death (1995)', 'genres': 'Action'}, 10: {'title': 'GoldenEye (1995)', 'genres': 'Action|Adventure|Thriller'}, 11: {'title': 'American President, The (1995)', 'genres': 'Comedy|Drama|Romance'}, 12: {'title': 'Dracula: Dead and Loving It (1995)', 'genres': 'Comedy|Horror'}, 13: {'title': 'Balto (1995)', 'genres': 'Adventure|Animation|Children'}, 14: {'title': 'N

### 统计每个电影平均评分

In [12]:
g = ratings.groupby('movieId').mean()
indexes = g.index # 行号-索引对照
for i in range(g.shape[0]):
    # print(indexes[i],g.iloc[i,1])
    if indexes[i] not in mv:
        mv[indexes[i]] = dict()
    mv[indexes[i]]['meanRating'] = g.iloc[i,1]
print(mv)

{1: {'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'tag': ['Pixar'], 'meanRating': 3.8724696356275303}, 2: {'title': 'Jumanji (1995)', 'genres': 'Adventure|Children|Fantasy', 'meanRating': 3.4018691588785046}, 3: {'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance', 'meanRating': 3.1610169491525424}, 4: {'title': 'Waiting to Exhale (1995)', 'genres': 'Comedy|Drama|Romance', 'meanRating': 2.3846153846153846}, 5: {'title': 'Father of the Bride Part II (1995)', 'genres': 'Comedy', 'tag': ['steve martin'], 'meanRating': 3.267857142857143}, 6: {'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller', 'meanRating': 3.8846153846153846}, 7: {'title': 'Sabrina (1995)', 'genres': 'Comedy|Romance', 'meanRating': 3.2830188679245285}, 8: {'title': 'Tom and Huck (1995)', 'genres': 'Adventure|Children', 'meanRating': 3.8}, 9: {'title': 'Sudden Death (1995)', 'genres': 'Action', 'meanRating': 3.15}, 10: {'title': 'GoldenEye (1995)', 'genres': 'Actio

### 统计每个电影的imdbId和tmdbId

In [13]:
g = links
for i in range(g.shape[0]):
    if math.isnan(g.iloc[i,1]) or math.isnan(g.iloc[i,2]):
        continue
    index = g.iloc[i,0]
    if index not in mv:
        mv[index] = dict()
    #print(g.iloc[i,1],g.iloc[i,2])
    mv[index]['imdbId'] = -1 if (g.iloc[i,1] == np.nan) else int(g.iloc[i,1])
    mv[index]['tmdbId'] = -1 if (g.iloc[i,2] == np.nan) else int(g.iloc[i,2])
print(mv)

{1: {'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'tag': ['Pixar'], 'meanRating': 3.8724696356275303, 'imdbId': 114709, 'tmdbId': 862}, 2: {'title': 'Jumanji (1995)', 'genres': 'Adventure|Children|Fantasy', 'meanRating': 3.4018691588785046, 'imdbId': 113497, 'tmdbId': 8844}, 3: {'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance', 'meanRating': 3.1610169491525424, 'imdbId': 113228, 'tmdbId': 15602}, 4: {'title': 'Waiting to Exhale (1995)', 'genres': 'Comedy|Drama|Romance', 'meanRating': 2.3846153846153846, 'imdbId': 114885, 'tmdbId': 31357}, 5: {'title': 'Father of the Bride Part II (1995)', 'genres': 'Comedy', 'tag': ['steve martin'], 'meanRating': 3.267857142857143, 'imdbId': 113041, 'tmdbId': 11862}, 6: {'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller', 'meanRating': 3.8846153846153846, 'imdbId': 113277, 'tmdbId': 949}, 7: {'title': 'Sabrina (1995)', 'genres': 'Comedy|Romance', 'meanRating': 3.2830188679245285, 'imdbId': 1

### 保存为Pickle

In [15]:
import pickle

with open('movie.pkl','wb') as f:
    pickle.dump(mv,f)